# NeRF Quick Start Notebook

Plank-ing Hyundong 3D Reconstruction Project
Created 2022.07.05 <br>

There are 4 Steps in this notebook.<br>
1. Sampling images from video
2. Get camera poses with COLMAP
3. Run NeRF
4. Get mesh file


__You can skip step 1, 2 by revising the directory path at Step 3__

## Directory Structure

- PlankHyundong
    - notebook
        - __nerf_quick_start.ipynb__
        - nerf_colab.ipynb
        - nerf_wandb_colab.ipynb
        - colmap_colab.ipynb
    - utils
    - data
        - video
            - video.mp4
        - base_images
            - images
                - image1.jpg
                - image2.jpg
                - ...
            - pose.npy
        - custom_images
            - images
            

> 'base_images' folder is for who want to skip step 1, 2
<br><br>
> 'custom_images' folder is for who want to execute from step 1.
            

--------

## Step 1
### Video Sampling

In [ ]:
import cv2

# Video file path
data_path = '/PlankHyundong/data/video/video.mp4'
save_path = '/PlankHyundong/data/custom_images'

# Set the number of frame
frame = 50

vidcap = cv2.VideoCapture(dir)
                
cnt, num = 0, 1 # cnt -> Input frame #, num -> output Frame #.

total_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
cycle = int(total_length / frame) # calculate cycle

while vidcap.isOpened():
    ret,image = vidcap.read()
    if num > frame:
        break
    if ret and cnt % cycle == 0:  
        
        try:
            cv2.imwrite(f"{save_path}/image{num}.jpg", image)
            num+=1
        except:
            print("fail")
            
    cnt += 1
    
vidcap.release()


## 

## Step 2
### Run COLMAP to get camera pose

### Install dependent packages

In [ ]:

!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev

### Install Ceres-solver

- It takes 10 ~ 20 minutes..

In [ ]:
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd ceres-solver
!git checkout $(git describe --tags) # Checkout the latest release
%mkdir build
%cd build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make
!sudo make install

### Install COLMAP
- It takes 10 ~ 20 minutes

In [ ]:
!sudo apt-get install libmetis-dev # https://github.com/colmap/colmap/issues/1469

## Run in Google Colab & Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd /content
!git clone https://github.com/Fyusion/LLFF

In [ ]:
%cd /content/LLFF
!python imgs2poses.py /content/PlankHyundong/data/custom_images

## Step 3
### Run NeRF

- We used tensorflow 1.x

In [ ]:
try:
    %tensorflow_version 1.x
except ValueError:
    # 만약 %tensorflow_version 1.x magic 명령어가 작동하지 않는 경우
    !pip uninstall --yes tensorflow
    !pip install tensorflow==1.15
    import tensorflow
    print(tensorflow.__version__)

- install dependent-packages

In [ ]:
!sudo apt -qq install imagemagick
!pip install ConfigArgParse -qqq
!pip install imageio-ffmpeg -qqq

- clone NeRF source code

In [ ]:
!git clone https://github.com/ProtossDragoon/nerf-wandb.git

In [ ]:
%cd nerf-wandb
!ls -al

- Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

### Setting directory path

### Setting the Hyperparameter

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

dataset_name = 'hyundong360_100'
downsample_factor = 64 #@param {type:"slider", min:1, max:64, step:1}
netdepth = 4 #@param {type:"slider", min:4, max:16, step:2}
netwidth = 64 #@param {type:"slider", min:64, max:256, step:4}
experiment_name = f'{dataset_name}_{downsample_factor}_downsampled_{now}'
max_iter = 30000 #@param
learning_rate = 0.01 #@param
video_saving_cnt = 3 #@param {type:"slider", min:1, max:10, step:1}
n_samples = 128 #@param {type:"slider", min:32, max:256, step:32}

# fine 모델에서 사용되는 샘플 개수는 coarse 모델의 sampling 개수의 2배로 설정한다.
# 공식 논문에서 제안하는 대로, 64이면 128.
n_importance = n_samples * 2

# Reproduce 를 위해 고정 random_seed 를 사용
random_seed = 777 #@param

# tradeoff: memory <-> speed (training 에는 속도와 성능 모두에 영향을 미치지 않음. 학습 도중 동영상을 만들 때 OOM 이 난다면 충분히 낮출 것)
rendering_speed = 2048 #@param {type:"slider", min:1024, max:16384, step:1024}

# tradeoff: memory <-> result
n_points_per_ray = 65536 #@param {type:"slider", min:2048, max:262144, step:1024}

_dummy_dir = f'./logs/{experiment_name}'
_tensorboard_logdir = f'./logs/summaries/{experiment_name}'
print(f'experiment: {experiment_name}')

In [ ]:
!python run_nerf.py \
    --wandbproject {'plank-hyundong'} \
    --wandbentity {'plank-hyundong'} \
    --maxiter {max_iter} \
    --datadir /content/drive/MyDrive/dev/llff_data/{dataset_name} \
    --dataset_type llff \
    --factor {downsample_factor} \
    --netdepth {netdepth} \
    --netwidth {netwidth} \
    --netdepth_fine {netdepth} \
    --netwidth_fine {netwidth} \
    --chunk {rendering_speed} \
    --netchunk {n_points_per_ray} \
    --lrate {learning_rate} \
    --i_video {max_iter // video_saving_cnt} \
    --expname {experiment_name} \
    --N_samples {n_samples} \
    --N_importance {n_importance} \
    --random_seed {random_seed} \
    --raw_noise_std 1.0 \
    --use_viewdirs \
    --no_ndc \
    --spherify \
    --lindisp \

## Step 4
### Get Mesh file